In [1]:
# import the necessary libraries
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from tqdm import tqdm
import numpy as np
import pandas as pd
pd.options.display.max_columns = None

In [5]:
#reading in the file into pandas dataframe
df = pd.read_csv('data/train.csv')

In [6]:
df.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


In [10]:
# total of 100 hotel clusters
df.hotel_cluster.nunique()

100

In [39]:
# separating the target variable and the predictor variables
df_X = df.drop(columns=['hotel_cluster'])
df_y = df['hotel_cluster']

In [45]:
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.05, random_state=0)

In [52]:
# using stratified shuffle split to get 5% of the original data while keeping the distribution
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.05, random_state=0)
for train_idx, test_idx in sss.split(df_X, df_y):
    X_train, X_test = df_X.iloc[train_idx], df_X.iloc[test_idx]
    y_train, y_test = df_y.iloc[train_idx], df_y.iloc[test_idx]
    break

In [53]:
# 5% of the original is about 1.8 mil samples
X_train.shape

(1883514, 23)

In [183]:
# Checking the distributions of eah target label
for idx in range(0, 100):
    print(f"{idx}: {100*(X_train.loc[(X_train['y'] == idx)].shape[0]/len(X_train))}")

0: 1.0084873274103618
1: 1.2017431248188226
2: 1.1534822677187428
3: 0.5979249424214527
4: 0.9339988978048478
5: 1.6463907356143888
6: 1.0204330841183022
7: 0.6701304051894491
8: 0.87883604794018
9: 1.296300425693677
10: 1.0744809966902291
11: 0.9304948091705185
12: 0.6875977561090599
13: 0.8918967419408615
14: 0.5104820033193276
15: 0.8604661287359692
16: 1.2685331778792193
17: 0.9273623663004363
18: 1.447507159490187
19: 0.750989905039198
20: 0.8091790132698775
21: 1.4603023922306921
22: 0.7703154847800441
23: 0.6881817708814482
24: 0.43567502020160187
25: 1.408537446496283
26: 1.0219727594273258
27: 0.27884050768935087
28: 1.3459416813466745
29: 1.0843030633167579
30: 1.2988488537913707
31: 0.683775113962519
32: 0.7705278537881853
33: 1.0818077274710993
34: 0.8272303789618766
35: 0.36930970515748757
36: 1.1319268133924145
37: 1.3168471272313347
38: 0.714727896899094
39: 0.9111692294296724
40: 1.1513585776373312
41: 2.0513253418875568
42: 1.4642843111333392
43: 0.6731566635554607
44:

### Missing Values

In [243]:
X_train.shape

(1883514, 25)

In [244]:
# trying to determine which columns have missing values
for idx in tqdm(range(X_train.shape[1])):
    print(f"Column{idx} has {X_train[X_train.iloc[:,idx].isna()].shape[0]} NaN values")

  4%|▍         | 1/25 [00:00<00:06,  3.84it/s]

Column0 has 0 NaN values
Column1 has 0 NaN values
Column2 has 0 NaN values
Column3 has 0 NaN values
Column4 has 0 NaN values
Column5 has 0 NaN values


 28%|██▊       | 7/25 [00:03<00:06,  3.00it/s]

Column6 has 675772 NaN values
Column7 has 0 NaN values
Column8 has 0 NaN values
Column9 has 0 NaN values
Column10 has 0 NaN values


 48%|████▊     | 12/25 [00:03<00:03,  3.87it/s]

Column11 has 2398 NaN values


100%|██████████| 25/25 [00:04<00:00,  6.07it/s]

Column12 has 2398 NaN values
Column13 has 0 NaN values
Column14 has 0 NaN values
Column15 has 0 NaN values
Column16 has 0 NaN values
Column17 has 0 NaN values
Column18 has 0 NaN values
Column19 has 0 NaN values
Column20 has 0 NaN values
Column21 has 0 NaN values
Column22 has 0 NaN values
Column23 has 0 NaN values
Column24 has 0 NaN values


### Date time Feature Engineering

In [377]:
#reset the index
sample_x = X_train.reset_index(drop=True)
sample_y = y_train.reset_index(drop=True)
sample_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883514 entries, 0 to 1883513
Data columns (total 25 columns):
date_time                    datetime64[ns]
site_name                    int64
posa_continent               int64
user_location_country        int64
user_location_region         int64
user_location_city           int64
orig_destination_distance    float64
user_id                      int64
is_mobile                    int64
is_package                   int64
channel                      int64
srch_ci                      object
srch_co                      object
srch_adults_cnt              int64
srch_children_cnt            int64
srch_rm_cnt                  int64
srch_destination_id          int64
srch_destination_type_id     int64
is_booking                   int64
cnt                          int64
hotel_continent              int64
hotel_country                int64
hotel_market                 int64
y                            int64
year                         int64

In [378]:
sample_x.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,y,year
0,2014-07-12 15:20:18,2,3,66,348,15528,1222.8908,1076811,0,1,3,2014-08-07,2014-08-10,1,1,1,28514,6,0,1,2,50,1457,2,2014
1,2014-07-20 16:44:53,11,3,205,354,53713,357.8904,395445,0,0,9,2014-08-29,2014-09-03,1,0,1,8267,1,0,1,2,50,675,21,2014
2,2014-05-08 11:39:39,2,3,66,220,49675,8842.4676,1101058,0,0,9,2015-01-04,2015-01-07,1,0,1,8227,1,1,1,3,104,67,81,2014
3,2013-07-13 15:50:49,22,2,0,203,54226,NaN,689933,0,0,9,2013-07-27,2013-07-28,1,0,1,12401,5,0,5,3,104,67,57,2013
4,2013-12-29 15:40:15,13,1,46,171,54827,NaN,251860,0,0,1,2014-10-22,2014-10-26,2,0,1,22532,6,0,1,0,63,1023,9,2013


In [379]:
# How many samples are greater than 2017? seems to be only one sample that has the year of 2557 in this 5% sample
#sample_x.loc[(sample_x['srch_ci'] > '2017')] # erroneous year
sample_x.loc[558005, 'srch_ci'] = '2014-11-14'
sample_x.loc[558005, 'srch_co'] = '2014-11-15'

# feature engineering into datetime
sample_x['date_time'] = pd.to_datetime(sample_x['date_time'], yearfirst=True)
sample_x['srch_ci'] = pd.to_datetime(sample_x['srch_ci'], yearfirst=True, errors='coerce')
sample_x['srch_co'] = pd.to_datetime(sample_x['srch_co'], yearfirst=True, errors='coerce')

# timedelta for days in advance feature
sample_x['advance'] = sample_x['srch_ci'] - sample_x['date_time']
sample_x['advance'] = sample_x.advance.dt.days

# duration of stays
sample_x['duration'] = sample_x['srch_co'] - sample_x['srch_ci']
sample_x['duration'] = sample_x.duration.dt.days

In [380]:
sample_x.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,y,year,advance,duration
0,2014-07-12 15:20:18,2,3,66,348,15528,1222.8908,1076811,0,1,3,2014-08-07,2014-08-10,1,1,1,28514,6,0,1,2,50,1457,2,2014,25.0,3.0
1,2014-07-20 16:44:53,11,3,205,354,53713,357.8904,395445,0,0,9,2014-08-29,2014-09-03,1,0,1,8267,1,0,1,2,50,675,21,2014,39.0,5.0
2,2014-05-08 11:39:39,2,3,66,220,49675,8842.4676,1101058,0,0,9,2015-01-04,2015-01-07,1,0,1,8227,1,1,1,3,104,67,81,2014,240.0,3.0
3,2013-07-13 15:50:49,22,2,0,203,54226,NaN,689933,0,0,9,2013-07-27,2013-07-28,1,0,1,12401,5,0,5,3,104,67,57,2013,13.0,1.0
4,2013-12-29 15:40:15,13,1,46,171,54827,NaN,251860,0,0,1,2014-10-22,2014-10-26,2,0,1,22532,6,0,1,0,63,1023,9,2013,296.0,4.0


In [381]:
sample_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883514 entries, 0 to 1883513
Data columns (total 27 columns):
date_time                    datetime64[ns]
site_name                    int64
posa_continent               int64
user_location_country        int64
user_location_region         int64
user_location_city           int64
orig_destination_distance    float64
user_id                      int64
is_mobile                    int64
is_package                   int64
channel                      int64
srch_ci                      datetime64[ns]
srch_co                      datetime64[ns]
srch_adults_cnt              int64
srch_children_cnt            int64
srch_rm_cnt                  int64
srch_destination_id          int64
srch_destination_type_id     int64
is_booking                   int64
cnt                          int64
hotel_continent              int64
hotel_country                int64
hotel_market                 int64
y                            int64
year              

In [382]:
new_x = sample_x.drop(columns=['hotel_market', 'user_id', 'site_name',
                      'posa_continent', 'user_location_region', 'user_location_city',
                     'hotel_continent', 'date_time', 'srch_ci', 'srch_co', 'y', 'year'])

In [385]:
new_x

,user_location_country,orig_destination_distance,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_country,advance,duration
0,66,1222.8908,0,1,3,1,1,1,28514,6,0,1,50,25.0,3.0
1,205,357.8904,0,0,9,1,0,1,8267,1,0,1,50,39.0,5.0
2,66,8842.4676,0,0,9,1,0,1,8227,1,1,1,104,240.0,3.0
3,0,NaN,0,0,9,1,0,1,12401,5,0,5,104,13.0,1.0
4,46,NaN,0,0,1,2,0,1,22532,6,0,1,63,296.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883509,205,5396.6495,0,0,5,4,0,2,8785,1,1,1,105,263.0,1.0
1883510,66,NaN,1,0,9,2,1,1,9145,1,0,2,50,24.0,2.0
1883511,69,NaN,0,0,9,1,1,1,12598,5,1,1,151,9.0,3.0
1883512,69,NaN,0,0,2,2,0,1,8407,6,0,1,144,154.0,5.0


In [389]:
# Train test split the 5% data
xtrain, xtest, ytrain, ytest = train_test_split(new_x, sample_y, test_size=.2 ,random_state=42)

In [391]:
xtrain.shape, xtest.shape

((1506811, 15), (376703, 15))

In [395]:
# Building the pipeline
def make_pipeline(clf=None):
    """This function here makes the pipeline but does not include the clf to prevent data leakage."""
    
    date_features = ['advance', 'duration']
    date_transformer = Pipeline(steps=[('imp', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0))])

    distance = ['orig_destination_distance']
    distance_transformer = Pipeline(steps=[('imp', SimpleImputer(missing_values=np.nan, strategy='mean'))])

    preprocessor = ColumnTransformer(transformers=[('date', date_transformer, date_features),
                                                   ('dist', distance_transformer, distance)],
                                    remainder='passthrough')

    pipeline = Pipeline([('preprocess', preprocessor)])
    
    return pipeline

In [404]:
def make_random_cv(cv, n_iter):
    """
    Steps:
    1. Define Estimator
    2. Define hyperparameter search space
    3. Instantiate RandomizedSearchCV
    """
    
    # YOUR CODE HERE
    rf = RandomForestClassifier(random_state=42)
    params_grid = {'n_estimators': [10, 100, 200, 400, 800, 1200],
                  'max_features': [2, 4, 6, 8],
                  'min_samples_leaf': [5, 10, 15, 20, 25]}
    
    clf_random_cv = RandomizedSearchCV(estimator=rf, scoring='f1_weighted', param_distributions=params_grid, n_iter=n_iter, cv=cv)
    
    return clf_random_cv

In [409]:
# make the pipeline and transform the data
pipeline = make_pipeline()
trans_xtrain = pipeline.fit_transform(xtrain)
trans_xtrain.shape # sanity check of datashape

(1506811, 15)

In [ ]:
# creating the RandomizedSearchCV
randomize_clf = make_random_cv(5, 10)
randomize_clf.fit(trans_xtrain, ytrain)

# Sandbox

# TOY SET
# taking about 6 samples to test where one sample has a messed up date
samples = X_train.head()
samples = samples.append(X_train[X_train['srch_ci'] == '2557-11-14'])
samples['year'] = samples.date_time.dt.year

# loop to find the wrong years
bad = []
for idx, row in samples.iterrows():
    t = []
    if row['srch_ci'].split('-')[0] not in ['2013', '2014', '2015']:
        t = samples.loc[idx, 'srch_ci'].split('-')
        t[0] = str(samples.loc[idx, 'year'])
        samples.loc[idx, 'srch_ci'] = '-'.join(t)

# ACTUAL SAMPLE SET
#bad = []
# This cell fixes the OutOfBound Error where the year was like 2557
#X_train_idx['date_time'] = pd.to_datetime(X_train_idx['date_time'])
#X_train_idx['year'] = X_train_idx.date_time.dt.year

for idx, row in tqdm(X_train_idx.iterrows()):
    try:
        if (row['srch_ci'].split('-')[0]) not in ['2013', '2014', '2015']:
            # checkin column
            t = X_train_idx.loc[idx, 'srch_ci'].split('-')
            t[0] = str(X_train_idx.loc[idx, 'year'])
            X_train_idx.loc[idx, 'srch_ci'] = '-'.join(t)

            # checkout column
            t = X_train_idx.loc[idx, 'srch_co'].split('-')
            t[0] = str(X_train_idx.loc[idx, 'year'])
            X_train_idx.loc[idx, 'srch_co'] = '-'.join(t)
            t = []
    except:
        continue
        #bad.append(idx)
# The ids was just to locate the NaN date rows for EDA
#ids = np.asarray(bad)

In [237]:
ids.shape

(2553,)

In [282]:
# Imputing the missing dates to 0 so that we can cover to datetime in pandas to feature engineer
imp = SimpleImputer(missing_values=0, strategy='constant', fill_value='0')
X_train_idx['srch_ci'] = imp.fit_transform(X_train_idx[['srch_ci']]).ravel()
X_train_idx['srch_co'] = imp.fit_transform(X_train_idx[['srch_co']]).ravel()

for idx, row in samples.iterrows():
    try:
        samples.loc[idx, 'advance'] = row['srch_ci'] - row['date_time']
    except:
        samples['srch_ci'].dt.strftime('%m%d')
        row['date_time'].dt.year
        samples.loc[idx, 'advance'] = row['srch_ci'] - row['date_time']
    

samples['srch_ci'].dt.strftime('%m%d')

In [293]:
# feature engineering into datetime
X_train_idx['date_time'] = pd.to_datetime(X_train_idx['date_time'], format='%d-%m-%Y', errors='coerce')
X_train_idx['srch_ci'] = pd.to_datetime(X_train_idx['srch_ci'], format='%d-%m-%Y', errors='coerce')
X_train_idx['srch_co'] = pd.to_datetime(X_train_idx['srch_co'], format='%d-%m-%Y', errors='coerce')

# timedelta for days in advance feature
#X_train['advance'] = X_train['srch_ci'] - X_train['date_time']
#X_train['advance'] = X_train.advance.dt.days

# duration of stays
#X_train['duration'] = X_train['srch_co'] - X_train['srch_ci']
#X_train['duration'] = X_train.duration.dt.days

In [167]:
X_train['year'] = X_train['date_time'].dt.year
#X_train[X_train['srch_ci'] == '2557-11-14']

/Users/dillonquan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [150]:
train_filter = X_train.drop(columns=['hotel_market', 'user_id', 'site_name',
                      'posa_continent', 'user_location_region', 'user_location_city',
                     'hotel_continent', 'date_time', 'srch_ci', 'srch_co'])

In [172]:
train_filter

,user_location_country,orig_destination_distance,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_country,y
34015300,66,1222.8908,0,1,3,1,1,1,28514,6,0,1,50,2
4796822,205,357.8904,0,0,9,1,0,1,8267,1,0,1,50,21
21461168,66,8842.4676,0,0,9,1,0,1,8227,1,1,1,104,81
15605930,0,NaN,0,0,9,1,0,1,12401,5,0,5,104,57
8290644,46,NaN,0,0,1,2,0,1,22532,6,0,1,63,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10628064,205,5396.6495,0,0,5,4,0,2,8785,1,1,1,105,64
10593935,66,NaN,1,0,9,2,1,1,9145,1,0,2,50,4
16739344,69,NaN,0,0,9,1,1,1,12598,5,1,1,151,78
13776549,69,NaN,0,0,2,2,0,1,8407,6,0,1,144,2


In [171]:
# trying to determine which columns have missing values
for idx in range(train_filter.shape[1]):
    print(f"Column{idx} has {train_filter[train_filter.iloc[:,idx].isna()].shape[0]} NaN values")

Column0 has 0 NaN values
Column1 has 675772 NaN values
Column2 has 0 NaN values
Column3 has 0 NaN values
Column4 has 0 NaN values
Column5 has 0 NaN values
Column6 has 0 NaN values
Column7 has 0 NaN values
Column8 has 0 NaN values
Column9 has 0 NaN values
Column10 has 0 NaN values
Column11 has 0 NaN values
Column12 has 0 NaN values
Column13 has 0 NaN values


### Holidays is TBD not sure how to set a datetime column without the years

In [147]:
samples['checkin'] = samples['srch_ci'].dt.strftime('%m%d')
#samples.checkin
#pd.to_datetime(df_sub['year'].astype(str)  + df_sub['Months'], format='%Y%B'
samples['checkin'] = pd.to_datetime(samples['checkin'], format='%m%d')

/Users/dillonquan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/dillonquan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


# redo this cell Nick's lesson
### Always assume that you cannot read in the data or even do EDA on the data. How can you access it?
import random
with open('data/train.csv', 'r') as f_handle:
    lst = []
    for x in range(38000000):
        ln = f_handle.readline()
        if x % 100000 == 0: print(x)
        if ln is None: break
        if random.random() < .1:
            lst.append( ln.split(',')[-1].replace('\n', ''))